<a href="https://colab.research.google.com/github/mobassir94/comprehensive-bangla-tts/blob/main/v2_Multilingual_(bn%2Bar)_tts_based_Qtafsir_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this notebook we will be using arabic fastspeech2 along with bangla hifi gan vocoder based GlowTTS for multilingual book reading purpose,we will try to read tafsir bayan multilingual (bangla+arabic+english) code mixed tafsir of https://www.hadithbd.com/ with the help of our multilingual TTS pipeline.

special thanks to 
1. [Arabic fastspeech2](https://github.com/ARBML/klaam)
2. [Bangla glowtts](https://github.com/Open-Speech-EkStep/vakyansh-tts)

Based on our previous [notebook](https://github.com/mobassir94/comprehensive-bangla-tts/blob/main/mlt_TTS_inference_demo/bn-ar-tts-pipeline.ipynb)

In [ ]:
%%capture
!pip install gdown==4.5.1
!pip install bnunicodenormalizer==0.1.1
# !pip install unidecode
# !pip install numpy
!pip install pydload==1.0.9
!pip install pydub

# Download and clone necessary files

In [ ]:

import gdown
import torch
import os
import numpy as np

from IPython.display import Audio
import soundfile as sf
import shutil

import pandas as pd
from pydub import AudioSegment
import sys

male = True

#https://www.kaggle.com/datasets/mobassir/bangla-quran-with-tafsir?select=ben_quran_with_tafsir.csv

q_tafsir = 'https://drive.google.com/drive/folders/1MJnVzKRCLzF_GLRkGALbHcLsHrNBl8xV?usp=sharing'
gdown.download_folder(url=q_tafsir, quiet=True)  


!git clone https://github.com/Open-Speech-EkStep/vakyansh-tts
!git clone https://github.com/ARBML/klaam



sys.path.append('/content/klaam')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #need cuda

device

Cloning into 'vakyansh-tts'...
remote: Enumerating objects: 1073, done.
remote: Counting objects: 100% (1073/1073), done.
remote: Compressing objects: 100% (411/411), done.
remote: Total 1073 (delta 650), reused 1070 (delta 649), pack-reused 0
Receiving objects: 100% (1073/1073), 402.46 KiB | 5.09 MiB/s, done.
Resolving deltas: 100% (650/650), done.
Cloning into 'klaam'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (584/584), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 759 (delta 276), reused 517 (delta 257), pack-reused 175
Receiving objects: 100% (759/759), 136.69 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (298/298), done.
Checking out files: 100% (175/175), done.


device(type='cuda')

In [ ]:
torch.__version__

'1.12.1+cu113'

In [ ]:
! echo $PYTHONPATH

/env/python


#Bangla GlowTTS

In [ ]:
%%capture
! pip install bnnumerizer
! pip install bangla==0.0.2
! pip install Unidecode
! pip install numpy


In [ ]:

os.chdir('vakyansh-tts') 

#skipping next 3 line of codes as we have already pytorch latest version installed in this environment

# !bash install.sh
# !python setup.py bdist_wheel
# !pip install -e .

os.chdir('tts_infer')
!mkdir translit_models
os.chdir('translit_models')
!wget https://storage.googleapis.com/vakyaansh-open-models/translit_models/default_lineup.json
!mkdir bangla
os.chdir('bangla')
!wget https://storage.googleapis.com/vakyaansh-open-models/translit_models/bengali/bengali_transliteration.zip
!unzip bengali_transliteration


if(male):
  print("male....")
  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/male_voice_1/glow.zip
  !unzip glow.zip

  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/male_voice_1/hifi.zip
  !unzip hifi.zip
else:
  print("female....")
  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/female_voice_0/glow.zip
  !unzip glow.zip

  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/female_voice_0/hifi.zip
  !unzip hifi.zip

!rm glow.zip
!rm hifi.zip




--2022-10-27 08:05:01--  https://storage.googleapis.com/vakyaansh-open-models/translit_models/default_lineup.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 209.85.145.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3422 (3.3K) [application/json]
Saving to: ‘default_lineup.json’

default_lineup.json 100%[===================>]   3.34K  --.-KB/s    in 0s      

2022-10-27 08:05:01 (58.5 MB/s) - ‘default_lineup.json’ saved [3422/3422]

--2022-10-27 08:05:02--  https://storage.googleapis.com/vakyaansh-open-models/translit_models/bengali/bengali_transliteration.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 209.85.145.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022

# Removing Annoying print statement during bangla TTS inference

In [ ]:
%%writefile /content/vakyansh-tts/tts_infer/transliterate.py

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
import sys
import os
import json
import enum
import traceback
import re

F_DIR = os.path.dirname(os.environ.get('translit_model_base_path', os.path.realpath(__file__)))


class XlitError(enum.Enum):
    lang_err = "Unsupported langauge ID requested ;( Please check available languages."
    string_err = "String passed is incompatable ;("
    internal_err = "Internal crash ;("
    unknown_err = "Unknown Failure"
    loading_err = "Loading failed ;( Check if metadata/paths are correctly configured."


##=================== Network ==================================================


class Encoder(nn.Module):
    def __init__(
        self,
        input_dim,
        embed_dim,
        hidden_dim,
        rnn_type="gru",
        layers=1,
        bidirectional=False,
        dropout=0,
        device="cpu",
    ):
        super(Encoder, self).__init__()

        self.input_dim = input_dim  # src_vocab_sz
        self.enc_embed_dim = embed_dim
        self.enc_hidden_dim = hidden_dim
        self.enc_rnn_type = rnn_type
        self.enc_layers = layers
        self.enc_directions = 2 if bidirectional else 1
        self.device = device

        self.embedding = nn.Embedding(self.input_dim, self.enc_embed_dim)

        if self.enc_rnn_type == "gru":
            self.enc_rnn = nn.GRU(
                input_size=self.enc_embed_dim,
                hidden_size=self.enc_hidden_dim,
                num_layers=self.enc_layers,
                bidirectional=bidirectional,
            )
        elif self.enc_rnn_type == "lstm":
            self.enc_rnn = nn.LSTM(
                input_size=self.enc_embed_dim,
                hidden_size=self.enc_hidden_dim,
                num_layers=self.enc_layers,
                bidirectional=bidirectional,
            )
        else:
            raise Exception("XlitError: unknown RNN type mentioned")

    def forward(self, x, x_sz, hidden=None):
        """
        x_sz: (batch_size, 1) -  Unpadded sequence lengths used for pack_pad
        """
        batch_sz = x.shape[0]
        # x: batch_size, max_length, enc_embed_dim
        x = self.embedding(x)

        ## pack the padded data
        # x: max_length, batch_size, enc_embed_dim -> for pack_pad
        x = x.permute(1, 0, 2)
        x = nn.utils.rnn.pack_padded_sequence(x, x_sz, enforce_sorted=False)  # unpad

        # output: packed_size, batch_size, enc_embed_dim
        # hidden: n_layer**num_directions, batch_size, hidden_dim | if LSTM (h_n, c_n)
        output, hidden = self.enc_rnn(
            x
        )  # gru returns hidden state of all timesteps as well as hidden state at last timestep

        ## pad the sequence to the max length in the batch
        # output: max_length, batch_size, enc_emb_dim*directions)
        output, _ = nn.utils.rnn.pad_packed_sequence(output)

        # output: batch_size, max_length, hidden_dim
        output = output.permute(1, 0, 2)

        return output, hidden

    def get_word_embedding(self, x):
        """ """
        x_sz = torch.tensor([len(x)])
        x_ = torch.tensor(x).unsqueeze(0).to(dtype=torch.long)
        # x: 1, max_length, enc_embed_dim
        x = self.embedding(x_)

        ## pack the padded data
        # x: max_length, 1, enc_embed_dim -> for pack_pad
        x = x.permute(1, 0, 2)
        x = nn.utils.rnn.pack_padded_sequence(x, x_sz, enforce_sorted=False)  # unpad

        # output: packed_size, 1, enc_embed_dim
        # hidden: n_layer**num_directions, 1, hidden_dim | if LSTM (h_n, c_n)
        output, hidden = self.enc_rnn(
            x
        )  # gru returns hidden state of all timesteps as well as hidden state at last timestep

        out_embed = hidden[0].squeeze()

        return out_embed


class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embed_dim,
        hidden_dim,
        rnn_type="gru",
        layers=1,
        use_attention=True,
        enc_outstate_dim=None,  # enc_directions * enc_hidden_dim
        dropout=0,
        device="cpu",
    ):
        super(Decoder, self).__init__()

        self.output_dim = output_dim  # tgt_vocab_sz
        self.dec_hidden_dim = hidden_dim
        self.dec_embed_dim = embed_dim
        self.dec_rnn_type = rnn_type
        self.dec_layers = layers
        self.use_attention = use_attention
        self.device = device
        if self.use_attention:
            self.enc_outstate_dim = enc_outstate_dim if enc_outstate_dim else hidden_dim
        else:
            self.enc_outstate_dim = 0

        self.embedding = nn.Embedding(self.output_dim, self.dec_embed_dim)

        if self.dec_rnn_type == "gru":
            self.dec_rnn = nn.GRU(
                input_size=self.dec_embed_dim
                + self.enc_outstate_dim,  # to concat attention_output
                hidden_size=self.dec_hidden_dim,  # previous Hidden
                num_layers=self.dec_layers,
                batch_first=True,
            )
        elif self.dec_rnn_type == "lstm":
            self.dec_rnn = nn.LSTM(
                input_size=self.dec_embed_dim
                + self.enc_outstate_dim,  # to concat attention_output
                hidden_size=self.dec_hidden_dim,  # previous Hidden
                num_layers=self.dec_layers,
                batch_first=True,
            )
        else:
            raise Exception("XlitError: unknown RNN type mentioned")

        self.fc = nn.Sequential(
            nn.Linear(self.dec_hidden_dim, self.dec_embed_dim),
            nn.LeakyReLU(),
            # nn.Linear(self.dec_embed_dim, self.dec_embed_dim), nn.LeakyReLU(), # removing to reduce size
            nn.Linear(self.dec_embed_dim, self.output_dim),
        )

        ##----- Attention ----------
        if self.use_attention:
            self.W1 = nn.Linear(self.enc_outstate_dim, self.dec_hidden_dim)
            self.W2 = nn.Linear(self.dec_hidden_dim, self.dec_hidden_dim)
            self.V = nn.Linear(self.dec_hidden_dim, 1)

    def attention(self, x, hidden, enc_output):
        """
        x: (batch_size, 1, dec_embed_dim) -> after Embedding
        enc_output: batch_size, max_length, enc_hidden_dim *num_directions
        hidden: n_layers, batch_size, hidden_size | if LSTM (h_n, c_n)
        """

        ## perform addition to calculate the score

        # hidden_with_time_axis: batch_size, 1, hidden_dim
        ## hidden_with_time_axis = hidden.permute(1, 0, 2) ## replaced with below 2lines
        hidden_with_time_axis = (
            torch.sum(hidden, axis=0)
            if self.dec_rnn_type != "lstm"
            else torch.sum(hidden[0], axis=0)
        )  # h_n

        hidden_with_time_axis = hidden_with_time_axis.unsqueeze(1)

        # score: batch_size, max_length, hidden_dim
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))

        # attention_weights: batch_size, max_length, 1
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)

        # context_vector shape after sum == (batch_size, hidden_dim)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        # context_vector: batch_size, 1, hidden_dim
        context_vector = context_vector.unsqueeze(1)

        # attend_out (batch_size, 1, dec_embed_dim + hidden_size)
        attend_out = torch.cat((context_vector, x), -1)

        return attend_out, attention_weights

    def forward(self, x, hidden, enc_output):
        """
        x: (batch_size, 1)
        enc_output: batch_size, max_length, dec_embed_dim
        hidden: n_layer, batch_size, hidden_size | lstm: (h_n, c_n)
        """
        if (hidden is None) and (self.use_attention is False):
            raise Exception(
                "XlitError: No use of a decoder with No attention and No Hidden"
            )

        batch_sz = x.shape[0]

        if hidden is None:
            # hidden: n_layers, batch_size, hidden_dim
            hid_for_att = torch.zeros(
                (self.dec_layers, batch_sz, self.dec_hidden_dim)
            ).to(self.device)
        elif self.dec_rnn_type == "lstm":
            hid_for_att = hidden[1]  # c_n

        # x (batch_size, 1, dec_embed_dim) -> after embedding
        x = self.embedding(x)

        if self.use_attention:
            # x (batch_size, 1, dec_embed_dim + hidden_size) -> after attention
            # aw: (batch_size, max_length, 1)
            x, aw = self.attention(x, hidden, enc_output)
        else:
            x, aw = x, 0

        # passing the concatenated vector to the GRU
        # output: (batch_size, n_layers, hidden_size)
        # hidden: n_layers, batch_size, hidden_size | if LSTM (h_n, c_n)
        output, hidden = (
            self.dec_rnn(x, hidden) if hidden is not None else self.dec_rnn(x)
        )

        # output :shp: (batch_size * 1, hidden_size)
        output = output.view(-1, output.size(2))

        # output :shp: (batch_size * 1, output_dim)
        output = self.fc(output)

        return output, hidden, aw


class Seq2Seq(nn.Module):
    """
    Class dependency: Encoder, Decoder
    """

    def __init__(
        self, encoder, decoder, pass_enc2dec_hid=False, dropout=0, device="cpu"
    ):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.pass_enc2dec_hid = pass_enc2dec_hid
        _force_en2dec_hid_conv = False

        if self.pass_enc2dec_hid:
            assert (
                decoder.dec_hidden_dim == encoder.enc_hidden_dim
            ), "Hidden Dimension of encoder and decoder must be same, or unset `pass_enc2dec_hid`"
        if decoder.use_attention:
            assert (
                decoder.enc_outstate_dim
                == encoder.enc_directions * encoder.enc_hidden_dim
            ), "Set `enc_out_dim` correctly in decoder"
        assert (
            self.pass_enc2dec_hid or decoder.use_attention
        ), "No use of a decoder with No attention and No Hidden from Encoder"

        self.use_conv_4_enc2dec_hid = False
        if (
            self.pass_enc2dec_hid
            and (encoder.enc_directions * encoder.enc_layers != decoder.dec_layers)
        ) or _force_en2dec_hid_conv:
            if encoder.enc_rnn_type == "lstm" or encoder.enc_rnn_type == "lstm":
                raise Exception(
                    "XlitError: conv for enc2dec_hid not implemented; Change the layer numbers appropriately"
                )

            self.use_conv_4_enc2dec_hid = True
            self.enc_hid_1ax = encoder.enc_directions * encoder.enc_layers
            self.dec_hid_1ax = decoder.dec_layers
            self.e2d_hidden_conv = nn.Conv1d(self.enc_hid_1ax, self.dec_hid_1ax, 1)

    def enc2dec_hidden(self, enc_hidden):
        """
        enc_hidden: n_layer, batch_size, hidden_dim*num_directions
        TODO: Implement the logic for LSTm bsed model
        """
        # hidden: batch_size, enc_layer*num_directions, enc_hidden_dim
        hidden = enc_hidden.permute(1, 0, 2).contiguous()
        # hidden: batch_size, dec_layers, dec_hidden_dim -> [N,C,Tstep]
        hidden = self.e2d_hidden_conv(hidden)

        # hidden: dec_layers, batch_size , dec_hidden_dim
        hidden_for_dec = hidden.permute(1, 0, 2).contiguous()

        return hidden_for_dec

    def active_beam_inference(self, src, beam_width=3, max_tgt_sz=50):
        """Search based decoding
        src: (sequence_len)
        """

        def _avg_score(p_tup):
            """Used for Sorting
            TODO: Dividing by length of sequence power alpha as hyperparam
            """
            return p_tup[0]

        import sys

        batch_size = 1
        start_tok = src[0]
        end_tok = src[-1]
        src_sz = torch.tensor([len(src)])
        src_ = src.unsqueeze(0)

        # enc_output: (batch_size, padded_seq_length, enc_hidden_dim*num_direction)
        # enc_hidden: (enc_layers*num_direction, batch_size, hidden_dim)
        enc_output, enc_hidden = self.encoder(src_, src_sz)

        if self.pass_enc2dec_hid:
            # dec_hidden: dec_layers, batch_size , dec_hidden_dim
            if self.use_conv_4_enc2dec_hid:
                init_dec_hidden = self.enc2dec_hidden(enc_hidden)
            else:
                init_dec_hidden = enc_hidden
        else:
            # dec_hidden -> Will be initialized to zeros internally
            init_dec_hidden = None

        # top_pred[][0] = Σ-log_softmax
        # top_pred[][1] = sequence torch.tensor shape: (1)
        # top_pred[][2] = dec_hidden
        top_pred_list = [(0, start_tok.unsqueeze(0), init_dec_hidden)]

        for t in range(max_tgt_sz):
            cur_pred_list = []

            for p_tup in top_pred_list:
                if p_tup[1][-1] == end_tok:
                    cur_pred_list.append(p_tup)
                    continue

                # dec_hidden: dec_layers, 1, hidden_dim
                # dec_output: 1, output_dim
                dec_output, dec_hidden, _ = self.decoder(
                    x=p_tup[1][-1].view(1, 1),  # dec_input: (1,1)
                    hidden=p_tup[2],
                    enc_output=enc_output,
                )

                ## π{prob} = Σ{log(prob)} -> to prevent diminishing
                # dec_output: (1, output_dim)
                dec_output = nn.functional.log_softmax(dec_output, dim=1)
                # pred_topk.values & pred_topk.indices: (1, beam_width)
                pred_topk = torch.topk(dec_output, k=beam_width, dim=1)

                for i in range(beam_width):
                    sig_logsmx_ = p_tup[0] + pred_topk.values[0][i]
                    # seq_tensor_ : (seq_len)
                    seq_tensor_ = torch.cat((p_tup[1], pred_topk.indices[0][i].view(1)))

                    cur_pred_list.append((sig_logsmx_, seq_tensor_, dec_hidden))

            cur_pred_list.sort(key=_avg_score, reverse=True)  # Maximized order
            top_pred_list = cur_pred_list[:beam_width]

            # check if end_tok of all topk
            end_flags_ = [1 if t[1][-1] == end_tok else 0 for t in top_pred_list]
            if beam_width == sum(end_flags_):
                break

        pred_tnsr_list = [t[1] for t in top_pred_list]

        return pred_tnsr_list


##===================== Glyph handlers =======================================


class GlyphStrawboss:
    def __init__(self, glyphs="en"):
        """list of letters in a language in unicode
        lang: ISO Language code
        glyphs: json file with script information
        """
        if glyphs == "en":
            # Smallcase alone
            self.glyphs = [chr(alpha) for alpha in range(97, 122 + 1)]
        else:
            self.dossier = json.load(open(glyphs, encoding="utf-8"))
            self.glyphs = self.dossier["glyphs"]
            self.numsym_map = self.dossier["numsym_map"]

        self.char2idx = {}
        self.idx2char = {}
        self._create_index()

    def _create_index(self):

        self.char2idx["_"] = 0  # pad
        self.char2idx["$"] = 1  # start
        self.char2idx["#"] = 2  # end
        self.char2idx["*"] = 3  # Mask
        self.char2idx["'"] = 4  # apostrophe U+0027
        self.char2idx["%"] = 5  # unused
        self.char2idx["!"] = 6  # unused

        # letter to index mapping
        for idx, char in enumerate(self.glyphs):
            self.char2idx[char] = idx + 7  # +7 token initially

        # index to letter mapping
        for char, idx in self.char2idx.items():
            self.idx2char[idx] = char

    def size(self):
        return len(self.char2idx)

    def word2xlitvec(self, word):
        """Converts given string of gyphs(word) to vector(numpy)
        Also adds tokens for start and end
        """
        try:
            vec = [self.char2idx["$"]]  # start token
            for i in list(word):
                vec.append(self.char2idx[i])
            vec.append(self.char2idx["#"])  # end token

            vec = np.asarray(vec, dtype=np.int64)
            return vec

        except Exception as error:
            print("XlitError: In word:", word, "Error Char not in Token:", error)
            sys.exit()

    def xlitvec2word(self, vector):
        """Converts vector(numpy) to string of glyphs(word)"""
        char_list = []
        for i in vector:
            char_list.append(self.idx2char[i])

        word = "".join(char_list).replace("$", "").replace("#", "")  # remove tokens
        word = word.replace("_", "").replace("*", "")  # remove tokens
        return word


class VocabSanitizer:
    def __init__(self, data_file):
        """
        data_file: path to file conatining vocabulary list
        """
        extension = os.path.splitext(data_file)[-1]
        if extension == ".json":
            self.vocab_set = set(json.load(open(data_file, encoding="utf-8")))
        elif extension == ".csv":
            self.vocab_df = pd.read_csv(data_file).set_index("WORD")
            self.vocab_set = set(self.vocab_df.index)
        else:
            print("XlitError: Only Json/CSV file extension supported")

    def reposition(self, word_list):
        """Reorder Words in list"""
        new_list = []
        temp_ = word_list.copy()
        for v in word_list:
            if v in self.vocab_set:
                new_list.append(v)
                temp_.remove(v)
        new_list.extend(temp_)

        return new_list


##=============== INSTANTIATION ================================================


class XlitPiston:
    """
    For handling prediction & post-processing of transliteration for a single language
    Class dependency: Seq2Seq, GlyphStrawboss, VocabSanitizer
    Global Variables: F_DIR
    """

    def __init__(
        self,
        weight_path,
        vocab_file,
        tglyph_cfg_file,
        iglyph_cfg_file="en",
        device="cpu",
    ):

        self.device = device
        self.in_glyph_obj = GlyphStrawboss(iglyph_cfg_file)
        self.tgt_glyph_obj = GlyphStrawboss(glyphs=tglyph_cfg_file)
        self.voc_sanity = VocabSanitizer(vocab_file)

        self._numsym_set = set(
            json.load(open(tglyph_cfg_file, encoding="utf-8"))["numsym_map"].keys()
        )
        self._inchar_set = set("abcdefghijklmnopqrstuvwxyz")
        self._natscr_set = set().union(
            self.tgt_glyph_obj.glyphs, sum(self.tgt_glyph_obj.numsym_map.values(), [])
        )

        ## Model Config Static                TODO: add defining in json support
        input_dim = self.in_glyph_obj.size()
        output_dim = self.tgt_glyph_obj.size()
        enc_emb_dim = 300
        dec_emb_dim = 300
        enc_hidden_dim = 512
        dec_hidden_dim = 512
        rnn_type = "lstm"
        enc2dec_hid = True
        attention = True
        enc_layers = 1
        dec_layers = 2
        m_dropout = 0
        enc_bidirect = True
        enc_outstate_dim = enc_hidden_dim * (2 if enc_bidirect else 1)

        enc = Encoder(
            input_dim=input_dim,
            embed_dim=enc_emb_dim,
            hidden_dim=enc_hidden_dim,
            rnn_type=rnn_type,
            layers=enc_layers,
            dropout=m_dropout,
            device=self.device,
            bidirectional=enc_bidirect,
        )
        dec = Decoder(
            output_dim=output_dim,
            embed_dim=dec_emb_dim,
            hidden_dim=dec_hidden_dim,
            rnn_type=rnn_type,
            layers=dec_layers,
            dropout=m_dropout,
            use_attention=attention,
            enc_outstate_dim=enc_outstate_dim,
            device=self.device,
        )
        self.model = Seq2Seq(enc, dec, pass_enc2dec_hid=enc2dec_hid, device=self.device)
        self.model = self.model.to(self.device)
        weights = torch.load(weight_path, map_location=torch.device(self.device))

        self.model.load_state_dict(weights)
        self.model.eval()

    def character_model(self, word, beam_width=1):
        in_vec = torch.from_numpy(self.in_glyph_obj.word2xlitvec(word)).to(self.device)
        ## change to active or passive beam
        p_out_list = self.model.active_beam_inference(in_vec, beam_width=beam_width)
        p_result = [
            self.tgt_glyph_obj.xlitvec2word(out.cpu().numpy()) for out in p_out_list
        ]

        result = self.voc_sanity.reposition(p_result)

        # List type
        return result

    def numsym_model(self, seg):
        """tgt_glyph_obj.numsym_map[x] returns a list object"""
        if len(seg) == 1:
            return [seg] + self.tgt_glyph_obj.numsym_map[seg]

        a = [self.tgt_glyph_obj.numsym_map[n][0] for n in seg]
        return [seg] + ["".join(a)]

    def _word_segementer(self, sequence):

        sequence = sequence.lower()
        accepted = set().union(self._numsym_set, self._inchar_set, self._natscr_set)
        # sequence = ''.join([i for i in sequence if i in accepted])

        segment = []
        idx = 0
        seq_ = list(sequence)
        while len(seq_):
            # for Number-Symbol
            temp = ""
            while len(seq_) and seq_[0] in self._numsym_set:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

            # for Target Chars
            temp = ""
            while len(seq_) and seq_[0] in self._natscr_set:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

            # for Input-Roman Chars
            temp = ""
            while len(seq_) and seq_[0] in self._inchar_set:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

            temp = ""
            while len(seq_) and seq_[0] not in accepted:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

        return segment

    def inferencer(self, sequence, beam_width=10):

        seg = self._word_segementer(sequence[:120])
        lit_seg = []

        p = 0
        while p < len(seg):
            if seg[p][0] in self._natscr_set:
                lit_seg.append([seg[p]])
                p += 1

            elif seg[p][0] in self._inchar_set:
                lit_seg.append(self.character_model(seg[p], beam_width=beam_width))
                p += 1

            elif seg[p][0] in self._numsym_set:  # num & punc
                lit_seg.append(self.numsym_model(seg[p]))
                p += 1
            else:
                lit_seg.append([seg[p]])
                p += 1

        ## IF segment less/equal to 2 then return combinotorial,
        ## ELSE only return top1 of each result concatenated
        if len(lit_seg) == 1:
            final_result = lit_seg[0]

        elif len(lit_seg) == 2:
            final_result = [""]
            for seg in lit_seg:
                new_result = []
                for s in seg:
                    for f in final_result:
                        new_result.append(f + s)
                final_result = new_result

        else:
            new_result = []
            for seg in lit_seg:
                new_result.append(seg[0])
            final_result = ["".join(new_result)]

        return final_result


from collections.abc import Iterable
from pydload import dload
import zipfile

MODEL_DOWNLOAD_URL_PREFIX = "https://github.com/AI4Bharat/IndianNLP-Transliteration/releases/download/xlit_v0.5.0/"


def is_folder_writable(folder):
    try:
        os.makedirs(folder, exist_ok=True)
        tmp_file = os.path.join(folder, ".write_test")
        with open(tmp_file, "w") as f:
            f.write("Permission Check")
        os.remove(tmp_file)
        return True
    except:
        return False


def is_directory_writable(path):
    if os.name == "nt":
        return is_folder_writable(path)
    return os.access(path, os.W_OK | os.X_OK)


class XlitEngine:
    """
    For Managing the top level tasks and applications of transliteration
    Global Variables: F_DIR
    """

    def __init__(
        self, lang2use="all", config_path="translit_models/default_lineup.json"
    ):

        lineup = json.load(open(os.path.join(F_DIR, config_path), encoding="utf-8"))
        self.lang_config = {}
        if isinstance(lang2use, str):
            if lang2use == "all":
                self.lang_config = lineup
            elif lang2use in lineup:
                self.lang_config[lang2use] = lineup[lang2use]
            else:
                raise Exception(
                    "XlitError: The entered Langauge code not found. Available are {}".format(
                        lineup.keys()
                    )
                )

        elif isinstance(lang2use, Iterable):
            for l in lang2use:
                try:
                    self.lang_config[l] = lineup[l]
                except:
                    print(
                        "XlitError: Language code {} not found, Skipping...".format(l)
                    )
        else:
            raise Exception(
                "XlitError: lang2use must be a list of language codes (or) string of single language code"
            )

        if is_directory_writable(F_DIR):
            models_path = os.path.join(F_DIR, "translit_models")
        else:
            user_home = os.path.expanduser("~")
            models_path = os.path.join(user_home, ".AI4Bharat_Xlit_Models")
        os.makedirs(models_path, exist_ok=True)
        self.download_models(models_path)

        self.langs = {}
        self.lang_model = {}
        for la in self.lang_config:
            try:
                #print("Loading {}...".format(la))
                self.lang_model[la] = XlitPiston(
                    weight_path=os.path.join(
                        models_path, self.lang_config[la]["weight"]
                    ),
                    vocab_file=os.path.join(models_path, self.lang_config[la]["vocab"]),
                    tglyph_cfg_file=os.path.join(
                        models_path, self.lang_config[la]["script"]
                    ),
                    iglyph_cfg_file="en",
                )
                self.langs[la] = self.lang_config[la]["name"]
            except Exception as error:
                print("XlitError: Failure in loading {} \n".format(la), error)
                print(XlitError.loading_err.value)

    def download_models(self, models_path):
        """
        Download models from GitHub Releases if not exists
        """
        for l in self.lang_config:
            lang_name = self.lang_config[l]["eng_name"]
            lang_model_path = os.path.join(models_path, lang_name)
            if not os.path.isdir(lang_model_path):
                print("Downloading model for language: %s" % lang_name)
                remote_url = MODEL_DOWNLOAD_URL_PREFIX + lang_name + ".zip"
                downloaded_zip_path = os.path.join(models_path, lang_name + ".zip")
                dload(url=remote_url, save_to_path=downloaded_zip_path, max_time=None)

                if not os.path.isfile(downloaded_zip_path):
                    exit(
                        f"ERROR: Unable to download model from {remote_url} into {models_path}"
                    )

                with zipfile.ZipFile(downloaded_zip_path, "r") as zip_ref:
                    zip_ref.extractall(models_path)

                if os.path.isdir(lang_model_path):
                    os.remove(downloaded_zip_path)
                else:
                    exit(
                        f"ERROR: Unable to find models in {lang_model_path} after download"
                    )
        return

    def translit_word(self, eng_word, lang_code="default", topk=7, beam_width=10):
        if eng_word == "":
            return []

        if lang_code in self.langs:
            try:
                res_list = self.lang_model[lang_code].inferencer(
                    eng_word, beam_width=beam_width
                )
                return res_list[:topk]

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        elif lang_code == "default":
            try:
                res_dict = {}
                for la in self.lang_model:
                    res = self.lang_model[la].inferencer(
                        eng_word, beam_width=beam_width
                    )
                    res_dict[la] = res[:topk]
                return res_dict

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        else:
            print("XlitError: Unknown Langauge requested", lang_code)
            print(XlitError.lang_err.value)
            return XlitError.lang_err

    def translit_sentence(self, eng_sentence, lang_code="default", beam_width=10):
        if eng_sentence == "":
            return []

        if lang_code in self.langs:
            try:
                out_str = ""
                for word in eng_sentence.split():
                    res_ = self.lang_model[lang_code].inferencer(
                        word, beam_width=beam_width
                    )
                    out_str = out_str + res_[0] + " "
                return out_str[:-1]

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        elif lang_code == "default":
            try:
                res_dict = {}
                for la in self.lang_model:
                    out_str = ""
                    for word in eng_sentence.split():
                        res_ = self.lang_model[la].inferencer(
                            word, beam_width=beam_width
                        )
                        out_str = out_str + res_[0] + " "
                    res_dict[la] = out_str[:-1]
                return res_dict

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        else:
            print("XlitError: Unknown Langauge requested", lang_code)
            print(XlitError.lang_err.value)
            return XlitError.lang_err


if __name__ == "__main__":

    available_lang = [
        "bn",
        "gu",
        "hi",
        "kn",
        "gom",
        "mai",
        "ml",
        "mr",
        "pa",
        "sd",
        "si",
        "ta",
        "te",
        "ur",
    ]

    reg = re.compile(r"[a-zA-Z]")
    lang = "hi"
    engine = XlitEngine(
        lang
    )  # if you don't specify lang code here, this will give results in all langs available
    sent = "Hello World! ABCD क्या हाल है आपका?"
    words = [
        engine.translit_word(word, topk=1)[lang][0] if reg.match(word) else word
        for word in sent.split()
    ]  # only transliterated en words, leaves rest as it is
    updated_sent = " ".join(words)

    print(updated_sent)

    # output : हेलो वर्ल्ड! क्या हाल है आपका?

    # y = engine.translit_sentence("Hello World !")['hi']
    # print(y)


Overwriting /content/vakyansh-tts/tts_infer/transliterate.py


# Bengali TTS inference demo

In [ ]:
# os.chdir('/content/vakyansh-tts/')

sys.path.append('/content/vakyansh-tts')
import bangla
from tts_infer.tts import TextToMel, MelToWav
from tts_infer.transliterate import XlitEngine
from tts_infer.num_to_word_on_sent import normalize_nums

import re
from scipy.io.wavfile import write

text_to_mel = TextToMel(glow_model_dir='/content/vakyansh-tts/tts_infer/translit_models/bangla/glow_ckp', device=device)
mel_to_wav = MelToWav(hifi_model_dir='/content/vakyansh-tts/tts_infer/translit_models/bangla/hifi_ckp', device=device)

def translit(text, lang):
    reg = re.compile(r'[a-zA-Z]')
    engine = XlitEngine(lang)
    words = [engine.translit_word(word, topk=1)[lang][0] if reg.match(word) else word for word in text.split()]
    updated_sent = ' '.join(words)
    return updated_sent
    
def run_tts(text, lang):
    # text = text.replace('।', '.') # only for hindi models
    text_num_to_word = normalize_nums(text, lang) # converting numbers to words in lang
    text_num_to_word_and_transliterated = translit(text_num_to_word, lang) # transliterating english words to lang
    
    mel = text_to_mel.generate_mel(text_num_to_word_and_transliterated)
    audio, sr = mel_to_wav.generate_wav(mel)
    write(filename='temp.wav', rate=sr, data=audio) # for saving wav file, if needed
    return (sr, audio)

def ben_tts(text, lang):
    # text = text.replace('।', '.') # only for hindi models
    text_num_to_word = normalize_nums(text, lang) # converting numbers to words in lang
    text_num_to_word_and_transliterated = translit(text_num_to_word, lang) # transliterating english words to lang
    
    mel = text_to_mel.generate_mel(text_num_to_word_and_transliterated)
    audio, sr = mel_to_wav.generate_wav(mel)
    del mel,text_num_to_word_and_transliterated,text_num_to_word
    return audio

/content/vakyansh-tts/tts_infer/translit_models/bangla/glow_ckp/G_200.pth


/content/vakyansh-tts/tts_infer/../src/glow_tts/modules.py:234: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2491.)
  w_init = torch.qr(torch.FloatTensor(self.n_split, self.n_split).normal_())[0]


/content/vakyansh-tts/tts_infer/translit_models/bangla/hifi_ckp/g_00100000
Loading '/content/vakyansh-tts/tts_infer/translit_models/bangla/hifi_ckp/g_00100000'
Complete.
Removing weight norm...


In [ ]:
!pwd

/content/vakyansh-tts/tts_infer/translit_models/bangla


In [ ]:
os.chdir('/content/') 

In [ ]:
!pwd

/content


In [ ]:
_, audio = run_tts('বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আতলু’ ফে’ল ক্রিয়া উহ্য আছে। অর্থাৎ, আল্লাহর নাম নিয়ে পড়ছি  কিংবা তেলাঅত আরম্ভ করছি। ', 'bn')
Audio('temp.wav')

In [ ]:
!nvidia-smi

Thu Oct 27 08:05:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    33W /  70W |   1202MiB / 15109MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Arabic FastSpeech2 TTS

In [ ]:

os.chdir('klaam') 


In [ ]:
%%capture
!pip install pyarabic

!pip install -r requirements.txt

In [ ]:
os.chdir('/content') 
from klaam import TextToSpeech
from pyarabic.araby import strip_diacritics
root_path = "/content/klaam/"
prepare_tts_model_path = "/content/klaam/cfgs/FastSpeech2/config/Arabic/preprocess.yaml"
model_config_path = "/content/klaam/cfgs/FastSpeech2/config/Arabic/model.yaml"
train_config_path = "/content/klaam/cfgs/FastSpeech2/config/Arabic/train.yaml"
vocoder_config_path = "/content/klaam/cfgs/FastSpeech2/model_config/hifigan/config.json"
speaker_pre_trained_path = "/content/klaam/data/model_weights/hifigan/generator_universal.pth.tar"

ar_model = TextToSpeech(prepare_tts_model_path, model_config_path, train_config_path, vocoder_config_path, speaker_pre_trained_path,root_path)


Downloading...
From: https://drive.google.com/uc?id=1J7ZP_q-6mryXUhZ-8j9-RIItz2nJGOIX
To: /content/model.pth.tar
100%|██████████| 418M/418M [00:02<00:00, 191MB/s]


Removing weight norm...


In [ ]:
text = "ٱلۡحَمۡدُ لِلَّهِ رَبِّ ٱلۡعَٰلَمِينَ"

ar_model.synthesize(text)

#sf.write('sound.wav', wave, 22050)
Audio('./sample.wav')

#TTS BigTextToAudio

long text to audio conversion 

In [ ]:
import re
import torchaudio.functional as F
import torchaudio
from bnnumerizer import numerize
import gc
from bnunicodenormalizer import Normalizer 
# initialize
bnorm=Normalizer()

# Create empty audio file of 1 second duration (purpose -> post processing)

audio = AudioSegment.silent(duration=1000)
sound = audio.set_frame_rate(audio.frame_rate*2)
sound.export("./empty.wav", format="wav")

#loading empty audio file of  1 second to append before and after each arabic chunk for increasing mlt reading rhythm.

empty_audio, rate_of_sample = torchaudio.load('/content/empty.wav')
empty_audio = empty_audio.flatten()

def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

class BigTextToAudio(object):
    
    def __init__(self,
                 ar_model,
                 bn_model,
                 ar_sample_rate=22050,
                 bn_sample_rate=22050,
                 out_sample_rate=22050,
                 attribution_dict={"সাঃ":"সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম",                   
                                  "আঃ":"আলাইহিস সালাম",
                                  "রাঃ":"রাদিআল্লাহু আনহু",
                                  "রহঃ":"রহমাতুল্লাহি আলাইহি",
                                  "রহিঃ":"রহিমাহুল্লাহ",
                                  "হাফিঃ":"হাফিযাহুল্লাহ",
                                  "বায়ান":"বাইআন",
                                  "দাঃবাঃ":"দামাত বারাকাতুহুম,দামাত বারাকাতুল্লাহ",
                                  "আয়াত" : "আইআত",#আইআত
                                  # "ওয়া" : "ওআ",
                                  # "ওয়াসাল্লাম"  : "ওআসাল্লাম",
                                  # "কেন"  : "কেনো",
                                  # "কোন" : "কোনো",
                                  # "বল"   : "বলো",
                                  # "চল"   : "চলো",
                                  # "কর"   : "করো",
                                  # "রাখ"   : "রাখো",
                                  "’"     :  "",
                                  "‘"     : "",
                                  # "য়"     : "অ",
                                  # "সম্প্রদায়" : "সম্প্রদাই",
                                  # "রয়েছে"   : "রইছে",
                                  # "রয়েছ"    : "রইছ",
                                   "/"   : " বাই ",
                                  },
                resample_params={"lowpass_filter_width": 64,
                                "rolloff": 0.9475937167399596,
                                "resampling_method": "kaiser_window",
                                "beta": 14.769656459379492}
                ):
        '''
            Instantiates a Big Text to Audio conversion object for bangla and arabic
            args:
                ar_model : arabic tts model
                bn_model : bangla tts model
                ar_sample_rate : arabic audio sample rate [optional] default: 22050
                bn_sample_rate : bangla audio sample rate [optional] default: 22050
                out_sample_rate : audio sample rate [optional] default: 22050
                attribution_dict : a dict for attribution expansion [optional]
                resample_params : audio resampling parameters [optional]
            resources:
                # Main class: modified from https://github.com/snakers4/silero-models/pull/174
                # Audio converter:https://www.kaggle.com/code/shahruk10/inference-notebook-wav2vec2
        '''
        self.ar_model = ar_model
        self.bn_model = bn_model
        self.attribution_dict=attribution_dict
 
        self.ar_sample_rate=ar_sample_rate
        self.bn_sample_rate=bn_sample_rate
        self.sample_rate=out_sample_rate  
        self.resample_params=resample_params
        
    # public
    def ar_tts(self,text):
        '''
            args: 
                text: arabic text (string)
            returns:
                audio as torch tensor
        '''
        text = strip_diacritics(text)

        try:
          self.ar_model.synthesize(text)
          audio, rate_of_sample = torchaudio.load('/content/sample.wav')
          audio = audio.flatten()
          audio = torch.cat([empty_audio,audio], axis=0) #start empty
          audio = torch.cat([audio,empty_audio], axis=0) #end empty
          audio = audio * 32768.0
        except:
            print("failed ar =>",text,"end")
            audio = empty_audio * 32768.0
                  
        
        return audio
    # public
    def bn_tts(self,text):
        '''
            args: 
                text   : bangla text (string)
            returns:
                audio as torch tensor
        '''
        if(self.bn_model == 'glowtts'):
          audio = ben_tts(text, 'bn')
          #audio = torch.from_numpy(audio)

        else:
          print("not supported!")

        return torch.tensor(audio)
    
    # public
    def expand_full_attribution(self,text):
        for word in self.attribution_dict:
            if word in text:
                text = text.replace(word, normalize(self.attribution_dict[word]))
        return text
    
    def collapse_whitespace(self,text):
        # Regular expression matching whitespace:
        _whitespace_re = re.compile(r"\s+")
        return re.sub(_whitespace_re, " ", text)

    # public
    def tag_text(self,text):
        '''
            * tags arabic text with <ar>text</ar>
            * tags bangla text with <bn>text</bn>
        '''
        # remove multiple spaces
        text=re.sub(' +', ' ',text)
        # create start and end
        text="start"+text+"end"
        # tag text
        parts=re.split(u'[\u0600-\u06FF]+', text)
        # remove non chars
        parts=[p for p in parts if p.strip()]
        # unique parts
        parts=set(parts)
        # tag the text
        for m in parts:
            if len(m.strip())>1:text=text.replace(m,f"</ar><SPLIT><bn>{m}</bn><SPLIT><ar>")
        # clean-tags
        text=text.replace("</ar><SPLIT><bn>start",'<bn>')
        text=text.replace("end</bn><SPLIT><ar>",'</bn>')
        return text

    def process_text(self,text):
        '''
        process multilingual text for suitable MLT TTS format
            * expand attributions
            * numerize text
            * tag sections of the text
            * sequentially list text blocks
            * Split based on sentence ending Characters

        '''
        
        # english numbers to bangla conversion
        res = re.search('[0-9]', text)
        if res is not None:
          text = bangla.convert_english_digit_to_bangla_digit(text)
        
        #replace ':' in between two bangla numbers with ' এর '
        pattern=r"[০, ১, ২, ৩, ৪, ৫, ৬, ৭, ৮, ৯]:[০, ১, ২, ৩, ৪, ৫, ৬, ৭, ৮, ৯]"
        matches=re.findall(pattern,text)
        for m in matches:
            r=m.replace(":"," এর ")
            text=text.replace(m,r)

        # numerize text
        text=numerize(text)

        # tag sections
        text=self.tag_text(text)
        # text blocks
        blocks=text.split("<SPLIT>")
        blocks=[b for b in blocks if b.strip()]
        # create tuple of (lang,text)
        data=[]
        for block in blocks:
            lang=None
            if "<bn>" in block:
                block=block.replace("<bn>",'').replace("</bn>",'')
                lang="bn"
            elif "<ar>" in block:
                block=block.replace("<ar>",'').replace("</ar>",'')
                lang="ar"
            
            # Split based on sentence ending Characters

            if lang == "bn":
              bn_text = block.strip()

              sentenceEnders = re.compile('[।!?]')
              sentences = sentenceEnders.split(str(bn_text))

              for i in range(len(sentences)):
                  res = re.sub('\n','',sentences[i])
                  res = normalize(res)
                  # expand attributes
                  res=self.expand_full_attribution(res)

                  res = self.collapse_whitespace(res)
                  res += '।'
                  data.append((lang,res))

            elif lang == "ar":
                ar_text = block.strip()
                ar_text = re.sub("؟", "?", ar_text) # replace any ؟ with ?

                sentenceEnders = re.compile('[.!?]')
                sentences = sentenceEnders.split(str(ar_text))

                for i in range(len(sentences)):
                    res = re.sub('\n','',sentences[i])
                    res = self.collapse_whitespace(res)
                    data.append((lang,res))
                    
        return data
    
    def resample_audio(self,audio,sr):
        '''
            resample audio with sample rate
            args:
                audio : torch.tensor audio
                sr: audi sample rate
        '''
        if sr==self.sample_rate:
            return audio
        else:
            return F.resample(audio,sr,self.sample_rate,**self.resample_params)
        
    
    def get_audio(self,data):
        '''
            creates audio from given data 
                * data=List[Tuples(lang,text)]
        '''
        audio_list = []
        for block in data:
            lang,text=block
            if lang=="bn":
                audio=self.bn_tts(text)
                sr=self.bn_sample_rate
            else:
                audio=self.ar_tts(text)
                sr=self.ar_sample_rate
            
            if self.resample_audio_to_out_sample_rate:
                audio=self.resample_audio(audio,sr)
                
            audio_list.append(audio)
  
        audio = torch.cat([k for k in audio_list])
        return audio
    
    # call
    def __call__(self,text,resample_audio_to_out_sample_rate=True):
        '''
            args: 
                text   : bangla text (string)
                resample_audio_to_out_sample_rate: for different sample rate in different models, resample the output audio 
                                                   in uniform sample rate 
                                                   * default:True
            returns:
                audio as numpy data
        '''
        self.resample_audio_to_out_sample_rate=resample_audio_to_out_sample_rate
        data=self.process_text(text)
        audio=self.get_audio(data)
        return audio.detach().cpu().numpy()

#check textprocessing

In [ ]:

df = pd.read_csv('/content/bn_qtafsir/ben_quran_with_tafsir.csv')
text = df.tafsir_bayan_text[1]
print(text)
bn_model = 'glowtts'
MLT_TTS=BigTextToAudio(ar_model=ar_model,
                   bn_model=bn_model)

data=MLT_TTS.process_text(text)
data

In [ ]:

audio=MLT_TTS(text,resample_audio_to_out_sample_rate=True)
Audio(audio, rate=MLT_TTS.sample_rate)

In [ ]:
sf.write(f'./check_mlt.wav', audio/ 32768.0, 22050)
#Audio('/content/check_mlt.wav', autoplay=True)

In [ ]:
len(df.tafsir_bayan_text)

6236

In [ ]:
def mlt_reader(surah_list,df):
    '''
      surah_list -> list of surah's for what we will be launching MLT TTS
      df         -> dataframe containing all bangla quranic verses and tafsir
    '''
    surah_curr=surah_list[0]
    bn_audio = []
    for i in range(len(df)):
        surah = df.ayat[i].split(",")[0]
        surah_num = int(re.findall(r'\d+', surah)[0])
        
        if(surah_num not in surah_list):
            continue
        else:
            bn_verse = df.ayat[i].replace("surah","সুরা").replace("ayat","আয়াত")
            bn_verse = bangla.convert_english_digit_to_bangla_digit(bn_verse)
            try:
              bn_qtafsir = bn_verse +" এর বাংলা অনুবাদ পরছি। "+ df.আল_বায়ান[i] + " । " + " আয়াত টির বাংলা তাফসির পরছি। "+df.tafsir_bayan_text[i] 
            except:
              #debug
              print("bn_verse ",bn_verse,"আল_বায়ান ",df.আল_বায়ান[i],"df.tafsir_bayan_text[i]  \n\n",df.tafsir_bayan_text[i] )
            audio=MLT_TTS(bn_qtafsir,resample_audio_to_out_sample_rate=True)
            # if current surah is bein
            if surah_curr==surah_num:    
                bn_audio.append(audio)

                full_surah_audio=np.concatenate(bn_audio,axis=-1)
                sf.write(f'./{surah_num}.wav', full_surah_audio/ 32768.0, 22050)
            else:
                bn_audio=[]
                surah_curr=surah_num

In [ ]:
df = df.replace(np.nan, 'এটির তাফসির নেই।', regex=True)

In [ ]:
%%time

surah_list = [1]
mlt_reader(surah_list = surah_list,df = df)

skipped
['<', 'a', 'l', 'H', 'm', 'd']
Raw Text Sequence: الحمد
Phoneme Sequence: {< a l H m d sil}
Raw Text Sequence: ال
Phoneme Sequence: {< a l sil}
Raw Text Sequence: استغراق
Phoneme Sequence: {s t g r AA q sil}
Raw Text Sequence: اختصاص
Phoneme Sequence: {x t S AA S sil}
skipped
['r', 'b']
Raw Text Sequence: رب
Phoneme Sequence: {r b sil}
Raw Text Sequence: عالمين عالم
Phoneme Sequence: {E aa l m ii0 n E aa l m sil}
Raw Text Sequence: عالم
Phoneme Sequence: {E aa l m sil}
Raw Text Sequence: رحما
Phoneme Sequence: {r H m aa sil}
Raw Text Sequence: فعلان
Phoneme Sequence: {f E l aa n sil}
Raw Text Sequence: رحيم
Phoneme Sequence: {r H ii0 m sil}
Raw Text Sequence: فعيل
Phoneme Sequence: {f E ii0 l sil}
skipped
['<', 'a', 'l', 'l', 'h', 'm']
Raw Text Sequence: اللهم
Phoneme Sequence: {< a l l h m sil}
failed ar =>  اجعلنا منهم end
skipped
['t', 'm', 'l', 'k']
skipped
['n', 'f', 's']
skipped
['l', 'n', 'f', 's']
skipped
['w', 'a', 'l', '<', 'm', 'r']
skipped
['y', 'uu0', 'm', '<', '*'

In [ ]:
!pwd

/content


In [ ]:
# from IPython.display import Audio 
# Audio('/content/1.wav', autoplay=True)